In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trec-2025/2025-starter-kit-main/.example.env
/kaggle/input/trec-2025/2025-starter-kit-main/main.py
/kaggle/input/trec-2025/2025-starter-kit-main/README.md
/kaggle/input/trec-2025/2025-starter-kit-main/produce_run.py
/kaggle/input/trec-2025/2025-starter-kit-main/output/tracking_data.json
/kaggle/input/trec-2025/2025-starter-kit-main/output/dragun-organizers-starter-kit-task-1
/kaggle/input/trec-2025/2025-starter-kit-main/output/dragun-organizers-starter-kit-task-2
/kaggle/input/trec-2025/2025-starter-kit-main/data/trec-2025-dragun-topics.jsonl
/kaggle/input/trec-2025/2025-starter-kit-main/modules/report_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/segment_retriever.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/question_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/query_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/information_evaluator.py


In [16]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth
    !pip install langchain langchain-community
    !pip install jsonlines
    !pip install --quiet langchain transformers sentencepiece

import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import json
from unsloth import FastLanguageModel
from unsloth import FastModel
import torch
from tqdm import tqdm
from unsloth.chat_templates import get_chat_template

In [17]:
TOPICS_PATH = '/kaggle/input/trec-2025/2025-starter-kit-main/data/trec-2025-dragun-topics.jsonl'

def load_jsonl(fname):
    with open(fname, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
topics = load_jsonl(TOPICS_PATH)
print("Loaded", len(topics), "topics.")

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)  

from langchain.prompts import PromptTemplate

Loaded 30 topics.
==((====))==  Unsloth 2025.7.11: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
# import re
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
# from langchain.llms import HuggingFacePipeline
# from transformers import pipeline

# # 1. Enhanced Prompt Template with few-shot examples and new ordering instruction
# prompt_template = PromptTemplate(
#     input_variables=["title", "body"],
#     template=(
#         "You are an expert research assistant. Read the news article below carefully.\n"
#         "Your task: Generate 10 critical investigative questions that a thoughtful reader should ask "
#         "to assess the article's trustworthiness, focusing on source bias, motivation, viewpoint diversity "
#         "(including centrist and right-wing perspectives), and factual accuracy.\n"
#         "Avoid vague or compound questions. Each question must be under 300 characters.\n\n"

#         "Rank the questions so that the most important and trust-assessing appear at the top.\n\n"

#         "Example 1:\n"
#         "Title: 'WHO: Processed meat causes cancer'\n"
#         "Article text: WHO has classified processed meat as carcinogenic...\n"
#         "Questions:\n"
#         "1. What research supports the WHO's classification of processed meat?\n"
#         "2. Who funded the cited studies on cancer and meat consumption?\n"
#         "3. Are alternative expert opinions presented in the article?\n"
#         "4. Is statistical data in the article properly attributed?\n"
#         "5. Does the article provide any counterarguments?\n"
#         "6. Are potential conflicts of interest disclosed?\n"
#         "7. Is there mention of population differences in risk?\n"
#         "8. Is sensational language used to exaggerate findings?\n"
#         "9. Does the article include primary sources or links?\n"
#         "10. Are the health claims consistent with other organizations?\n\n"

#         "Example 2:\n"
#         "Title: 'CDC says masks are optional outdoors'\n"
#         "Article text: New CDC guidelines suggest masks are no longer necessary...\n"
#         "Questions:\n"
#         "1. What scientific evidence is used to support this guideline?\n"
#         "2. Does the article quote public health officials directly?\n"
#         "3. Are there experts who disagree with the CDC's stance?\n"
#         "4. Are specific situations where masks are still needed mentioned?\n"
#         "5. Does the article specify data sources for transmission risk?\n"
#         "6. Are regional COVID-19 differences addressed?\n"
#         "7. Are the potential political implications discussed?\n"
#         "8. Is past CDC guidance compared to current updates?\n"
#         "9. Are international guidelines referenced?\n"
#         "10. Are any counterfactual scenarios analyzed?\n\n"

#         "Now generate 10 similar questions for the following article:\n\n"
#         "Title: {title}\n\n"
#         "Article text:\n{body}\n\n"
#         "Output only the questions as a numbered list:\n"
#     )
# )

# # 2. Pipeline
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=600,
#     temperature=0.7,
#     do_sample=True,
#     return_full_text=False
# )

# llm = HuggingFacePipeline(pipeline=pipe)

# llm_chain = LLMChain(
#     prompt=prompt_template,
#     llm=llm,
#     output_key="text"
# )

# # 3. Regex pattern for 1. Question format
# pattern = re.compile(r"^\s*\d+[.)]\s*(.+)$")

# # 4. Output container
# results = []

# # 5. Function to extract clean questions
# def extract_questions(raw_text):
#     questions = []
#     for line in raw_text.strip().split("\n"):
#         match = pattern.match(line.strip())
#         if match:
#             question = match.group(1).strip()
#             if 0 < len(question) <= 300:
#                 questions.append(question)
#     return questions

# # 6. Heuristic scoring function for ranking trustworthiness questions
# KEYWORDS = [
#     'bias', 'motivation', 'viewpoint', 'right-wing', 'centrist',
#     'diversity', 'balance', 'perspective', 'agenda', 'conflict of interest',
#     'credibility', 'trustworthiness', 'fairness', 'opinion', 'sources'
# ]

# def score_question(q):
#     q_lower = q.lower()
#     return sum(1 for kw in KEYWORDS if kw in q_lower)

# # 7. Main generation loop with intelligent retry and ranking
# for idx, topic in enumerate(topics):
#     docid = topic['docid']
#     title = topic.get('title', '').strip()
#     body = topic.get('body', '').strip()[:2000]

#     if not title and not body:
#         print(f"⚠️ Skipping empty topic {docid}")
#         continue

#     questions = []
#     for attempt in range(3):  # Up to 3 tries
#         output = llm_chain.invoke({"title": title, "body": body})
#         text = output['text'] if isinstance(output, dict) else output
#         questions = extract_questions(text)

#         if len(questions) >= 10:
#             # Sort by score, keep best 10
#             questions = sorted(questions, key=score_question, reverse=True)[:10]
#             break
#         else:
#             print(f"🔁 Retry {attempt + 1} for {docid} — Got {len(questions)} questions")

#     # Fill with 'N/A' if still less than 10 after retries
#     while len(questions) < 10:
#         questions.append("N/A")

#     results.append({
#         "docid": docid,
#         "questions": questions
#     })

# # 8. Print first 3 docs
# for item in results[:3]:
#     print(f"\nDocID: {item['docid']}")
#     for i, q in enumerate(item['questions'], 1):
#         print(f"{i}. {q}")

In [19]:
import re
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# 1. Enhanced Prompt Template with few-shot examples and new ordering instruction
prompt_template = PromptTemplate(
    input_variables=["title", "body"],
    template=(
        "You are an expert research assistant. Read the news article below carefully.\n"
        "Your task: Generate 10 critical investigative questions that a thoughtful reader should ask "
        "to assess the article's trustworthiness, focusing on source bias, motivation, viewpoint diversity "
        "(including centrist and right-wing perspectives), and factual accuracy.\n"
        "Avoid vague or compound questions. Each question must be under 300 characters.\n\n"

        "Rank the questions so that the most important and trust-assessing appear at the top.\n\n"

        "Example 1:\n"
        "Title: 'WHO: Processed meat causes cancer'\n"
        "Article text: WHO has classified processed meat as carcinogenic...\n"
        "Questions:\n"
        "1. What research supports the WHO's classification of processed meat?\n"
        "2. Who funded the cited studies on cancer and meat consumption?\n"
        "3. Are alternative expert opinions presented in the article?\n"
        "4. Is statistical data in the article properly attributed?\n"
        "5. Does the article provide any counterarguments?\n"
        "6. Are potential conflicts of interest disclosed?\n"
        "7. Is there mention of population differences in risk?\n"
        "8. Is sensational language used to exaggerate findings?\n"
        "9. Does the article include primary sources or links?\n"
        "10. Are the health claims consistent with other organizations?\n\n"

        "Example 2:\n"
        "Title: 'CDC says masks are optional outdoors'\n"
        "Article text: New CDC guidelines suggest masks are no longer necessary...\n"
        "Questions:\n"
        "1. What scientific evidence is used to support this guideline?\n"
        "2. Does the article quote public health officials directly?\n"
        "3. Are there experts who disagree with the CDC's stance?\n"
        "4. Are specific situations where masks are still needed mentioned?\n"
        "5. Does the article specify data sources for transmission risk?\n"
        "6. Are regional COVID-19 differences addressed?\n"
        "7. Are the potential political implications discussed?\n"
        "8. Is past CDC guidance compared to current updates?\n"
        "9. Are international guidelines referenced?\n"
        "10. Are any counterfactual scenarios analyzed?\n\n"

        "Now generate 10 similar questions for the following article:\n\n"
        "Title: {title}\n\n"
        "Article text:\n{body}\n\n"
        "Output only the questions as a numbered list:\n"
    )
)

# 2. Pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=600,
    temperature=0.7,
    do_sample=True,
    return_full_text=False
)
llm = HuggingFacePipeline(pipeline=pipe)
llm_chain = LLMChain(prompt=prompt_template, llm=llm, output_key="text")

# 3. Regex for extracting 1. ..., 2. ... format
pattern = re.compile(r"^\s*\d+[.)]\s*(.+)$")

# 4. Extraction function with de-duplication
def extract_unique_questions(raw_text):
    seen = set()
    questions = []
    for line in raw_text.strip().split("\n"):
        match = pattern.match(line.strip())
        if match:
            question = match.group(1).strip()
            if 0 < len(question) <= 300 and question not in seen:
                seen.add(question)
                questions.append(question)
    return questions

# 5. Main loop
results = []
for idx, topic in enumerate(topics):
    docid = topic['docid']
    title = topic.get('title', '').strip()
    body = topic.get('body', '').strip()[:2000]

    if not title and not body:
        print(f"⚠️ Skipping empty topic {docid}")
        continue

    questions = []
    for attempt in range(3):  # Retry 3 times if needed
        output = llm_chain.invoke({"title": title, "body": body})
        text = output['text'] if isinstance(output, dict) else output
        questions = extract_unique_questions(text)

        if len(questions) >= 10:
            questions = questions[:10]  # Truncate to 10 if more
            break
        else:
            print(f"🔁 Retry {attempt+1} for {docid} — Got {len(questions)} unique questions")

    # Fill with 'N/A' if less than 10
    while len(questions) < 10:
        questions.append("N/A")

    results.append({
        "docid": docid,
        "questions": questions
    })

# 6. Show a few examples
for item in results[:3]:
    print(f"\nDocID: {item['docid']}")
    for i, q in enumerate(item['questions'], 1):
        print(f"{i}. {q}")


🔁 Retry 1 for msmarco_v2.1_doc_34_7751734 — Got 0 unique questions
🔁 Retry 1 for msmarco_v2.1_doc_52_1666095905 — Got 0 unique questions

DocID: msmarco_v2.1_doc_04_420132660
1. What is the source of the claim that Sam Panopoulos invented the Hawaiian pizza?
2. Does the article cite any historical records or primary sources to support the invention story?
3. Are alternative explanations for the pizza's origin presented?
4. Are there any known conflicts of interest among the authors or contributors?
5. What is the perspective of Greek or Canadian historians on the pizza's origin?
6. Does the article address the cultural significance of the pizza in both Canada and Hawaii?
7. Are there any counterarguments or rebuttals from Hawaiian or Canadian cultural representatives?
8. What is the historical context of pizza in Canada and its evolution over time?
9. Does the article provide any evidence of the pizza's spread from Canada to Hawaii?
10. Are there any alternative theories about the pizz

In [20]:
import pandas as pd

team_id = "CUET"
run_id = "CUET-qwen4B-v3"

submission_rows = []

for item in results:
    topic_id = item["docid"]
    for rank, question in enumerate(item["questions"], 1):
        # Clean question of unwanted tabs/newlines
        clean_q = question.replace('\t', ' ').replace('\n', ' ').strip()
        submission_rows.append([
            topic_id,
            team_id,
            run_id,
            rank,
            clean_q
        ])

# Create DataFrame and save
df = pd.DataFrame(submission_rows, columns=["topic_id", "team_id", "run_id", "rank", "question"])
df.to_csv("/kaggle/working/CUET_run4.tsv", sep="\t", index=False, header=False, encoding="utf-8")

In [21]:
df[50:70]

,topic_id,team_id,run_id,rank,question
50,msmarco_v2.1_doc_22_1648697797,CUET,CUET-qwen4B-v3,1,What is the source of the allegations against ...
51,msmarco_v2.1_doc_22_1648697797,CUET,CUET-qwen4B-v3,2,Are the alleged texts or screenshots verified ...
52,msmarco_v2.1_doc_22_1648697797,CUET,CUET-qwen4B-v3,3,Does the article present any counterarguments ...
53,msmarco_v2.1_doc_22_1648697797,CUET,CUET-qwen4B-v3,4,Are the allegations consistent with any known ...
54,msmarco_v2.1_doc_22_1648697797,CUET,CUET-qwen4B-v3,5,What is the credibility of the individuals cit...
55,msmarco_v2.1_doc_22_1648697797,CUET,CUET-qwen4B-v3,6,Does the article mention any expert opinions o...
56,msmarco_v2.1_doc_22_1648697797,CUET,CUET-qwen4B-v3,7,Are there alternative explanations for the all...
57,msmarco_v2.1_doc_22_1648697797,CUET,CUET-qwen4B-v3,8,Are the allegations' implications on Armie Ham...
58,msmarco_v2.1_doc_22_1648697797,CUET,CUET-qwen4B-v3,9,Does the article provide context on the broade...
59,msmarco_v2.1_doc_22_1648697797,CUET,CUET-qwen4B-v3,10,Are there any potential conflicts of interest ...


In [25]:
# Show duplicated questions (exact text matches)
duplicates = df[df.duplicated('question', keep=False)]

print(f"Found {len(duplicates)} duplicated question rows")
duplicates

df['question'].value_counts()[df['question'].value_counts() > 1]


Found 0 duplicated question rows


Series([], Name: count, dtype: int64)